# 1. fasttext基本函数

## 1.1 加载fasttext词语向量并打印fasttext模型的帮助文件

fasttext使用C++写的，官网提供了Python的接口https://github.com/facebookresearch/fastText/tree/master/python 。还有一个使用 fastText的Python库叫做pyfasttext,但是它已经不再维护了https://github.com/vrasneur/pyfasttext 。

In [1]:
from fastText import load_model
fasttext_model = load_model(r'cc.zh.300.bin')
help(fasttext_model)

Help on _FastText in module fastText.FastText object:

class _FastText(builtins.object)
 |  _FastText(model=None)
 |  
 |  This class defines the API to inspect models and should not be used to
 |  create objects. It will be returned by functions such as load_model or
 |  train.
 |  
 |  In general this API assumes to be given only unicode for Python2 and the
 |  Python3 equvalent called str for any string-like arguments. All unicode
 |  strings are then encoded as UTF-8 and fed to the fastText C++ API.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, model=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  get_dimension(self)
 |      Get the dimension (size) of a lookup vector (hidden layer).
 |  
 |  get_input_matrix(self)
 |      Get a copy of the full input matrix of a Model. This only
 |      works if the model is not quantized.
 |  
 |  get_input_vector(self, ind)
 |      Given an index, get the corresponding vector of the Input Matrix.
 |

## 1.2 fasttext模型基本方法演示

In [2]:
# 获取词汇表
words = fasttext_model.get_words()
# 打印词汇表长度
print("词汇表长度=", len(words))
# 打印词汇表前100个词
print("词汇表前100个词\n", words[:100])
# 打印词语向量的维度
print("词语向量维度=",fasttext_model.get_dimension())
# 获取“是”字在词汇表中以及词语向量矩阵中的索引id
print('"是"字在词汇表中以及词语向量矩阵中的索引id', fasttext_model.get_word_id("是"))
# 通过id获取词语"是"向量
vec_by_id = fasttext_model.get_input_vector(5)
# 打印向量长度
import numpy as np
print('词语"是"的向量长度=', np.linalg.norm(vec_by_id))
# fasttext向量库中的词语向量不是单位向量，如果使用余弦相似度查找最相似的词，则需要事先对词语向量进行归一化处理
# 通过词语获取词语“是”的词语向量
vec_by_word = fasttext_model.get_word_vector("是")
print('比较两种获取词语"是"向量的方法是否一致?', np.linalg.norm(vec_by_id) == np.linalg.norm(vec_by_word))

词汇表长度= 2000000
词汇表前100个词
 ['，', '的', '。', '</s>', '、', '是', '一', '在', '：', '了', '（', '）', "'", '和', '不', '有', '我', ',', ')', '(', '“', '”', '也', '人', '个', ':', '中', '.', '就', '他', '》', '《', '-', '你', '都', '上', '大', '！', '这', '为', '多', '与', '章', '「', '到', '」', '要', '？', '被', '而', '能', '等', '可以', '年', '；', '|', '以', '及', '之', '公司', '对', '中国', '很', '会', '小', '但', '我们', '最', '更', '/', '1', '三', '新', '自己', '可', '2', '或', '次', '好', '将', '第', '种', '她', '…', '3', '地', '對', '用', '工作', '下', '后', '由', '两', '使用', '还', '又', '您', '?', '其', '已']
词语向量维度= 300
"是"字在词汇表中以及词语向量矩阵中的索引id 5
词语"是"的向量长度= 1.6197449
比较两种获取词语"是"向量的方法是否一致? True


# 2. 计算与"直升机"最相似的词语列表

## 2.1 首先对fasttext词语向量矩阵的每个词向量（行向量）归一化

In [3]:
# 获取整个fasttext向量矩阵
fasttext_vec = fasttext_model.get_input_matrix()[:len(fasttext_model.get_words())]
# 验证读取处的向量矩阵是否正确
input_vec = fasttext_vec[fasttext_model.get_word_id("的")]
model_vec = fasttext_model.get_word_vector("的")
print("input_vec与model_vec一致吗?", np.linalg.norm(input_vec) == np.linalg.norm(model_vec))
# 对fasttext向量矩阵的每个词向量（行向量）归一化
fasttext_vec = (fasttext_vec.T/np.linalg.norm(fasttext_vec,axis=1)).T
# 获取“的”的词语向量
word_vec = fasttext_vec[fasttext_model.get_word_id("的")]
# 输出“的”的向量长度
print(np.linalg.norm(word_vec))

input_vec与model_vec一致吗? True
1.0


## 2.2 检索与"直升机"相关的词

In [4]:
# 获取直升机的向量
word_vec = fasttext_vec[fasttext_model.get_word_id("直升机")]
# 计算直升机向量与库中每个词的相似度
sim_vec = np.dot(word_vec, fasttext_vec.T)
# 按相似度排序词语
sorted_sim_vec = sorted(zip(fasttext_model.get_words(), sim_vec), key=lambda x:x[1], reverse=True)
# 获取与直升机最相似的top10词语列表，
print(sorted_sim_vec[:10])
# 把词语直升机自己去除掉
print(sorted_sim_vec[1:10+1])

[('直升机', 1.0), ('直升飞机', 0.77804565), ('飞机', 0.71779704), ('直升機', 0.6616683), ('米-171', 0.6531244), ('米-8', 0.6311184), ('停机坪', 0.6279004), ('救援机', 0.6260574), ('NH-90', 0.62192786), ('Z-9', 0.6202519)]
[('直升飞机', 0.77804565), ('飞机', 0.71779704), ('直升機', 0.6616683), ('米-171', 0.6531244), ('米-8', 0.6311184), ('停机坪', 0.6279004), ('救援机', 0.6260574), ('NH-90', 0.62192786), ('Z-9', 0.6202519), ('旋翼机', 0.61378306)]
